In [0]:
%sql
SELECT MemoriaCalculo FROM desafio_kinea.boletagem_cp.nxenq_memoriacalculo
LIMIT 10

In [0]:
df = _sqldf.toPandas()

texts = list(df["MemoriaCalculo"])
print(texts[0])

In [0]:
def chega_final_texto(texto_memoria:str)->str:
    index1 = texto_memoria.find("MEMÓRIA DE CÁLCULO (PROCESSAMENTO)") #index do final da string, perto das colunas
    index_coluna = texto_memoria.find("TÍTULO",index1) #index da primeira coluna (TITULO)
    texto_final = texto_memoria[index_coluna:]

    linhas = texto_final.splitlines()
    apenas_linhas_dados = "\n".join(linhas[2:]) #pula linha de coluna e linhas tracejadas
    #print(apenas_linhas_dados)

    return apenas_linhas_dados


textos_processados = list(map(chega_final_texto,texts))

print(textos_processados[1])


prim_linha = textos_processados[3].splitlines()[0]
#print(len(prim_linha))
#print(prim_linha)

def printa_indice_comeco_coluna()->None:
    """
    Função para achar os índices de começo de cada coluna, utilizado para a variável colspecs do dataframe
    """
    is_word = False
    for i,char in enumerate(prim_linha):
        if is_word and not char.isspace():
            print(i)
            is_word = False
        if char.isspace():
            is_word = True
     

In [0]:
from io import StringIO
import pandas as pd

colspecs = [
    (0, 20),   # TÍTULO
    (21, 51),  # NOME
    (52, 72),  # POSIÇÃO
    (73, 83),  # %PL
    (84, 144), # GRUPO
    (144, None) # EMISSOR 
]

# Check if textos_processados[0] is not empty
if textos_processados[0].strip():
    print(textos_processados[0])

    # Check if the input is a single line
    if len(textos_processados[0].splitlines()) == 1:
        data = [textos_processados[0][start:end].strip() for start, end in colspecs]
        df = pd.DataFrame([data], columns=["TÍTULO", "NOME", "POSIÇÃO", "%PL", "GRUPO", "EMISSOR"])
    else:
        df = pd.read_fwf(
            StringIO(textos_processados[0]),
            colspecs=colspecs,
        )
        df.columns = ["TÍTULO", "NOME", "POSIÇÃO", "%PL", "GRUPO", "EMISSOR"]

    display(df)

    df = df.apply(lambda col: col.str.strip())

    def para_float_br(x):
        return float(x.replace('.', '').replace(',', '.'))

    df["POSIÇÃO"] = df["POSIÇÃO"].apply(para_float_br)
    df["%PL"] = df["%PL"].apply(para_float_br)

    display(df)
else:
    print("The dataset is empty.")

In [0]:
def texto_memoria_para_df(texto:str)->pd.DataFrame:
    final_texto = chega_final_texto(texto) #chega no final da string, onde tem as colunas de dados da memória
    colspecs = [
        (0, 20),   # TÍTULO
        (21, 51),  # NOME
        (52, 72),  # POSIÇÃO
        (73, 83),  # %PL
        (84, 144), # GRUPO
        (144, None) # EMISSOR 
    ]

    # Check if final_texto is not empty
    if final_texto.strip():
        # Check if the input is a single line
        if len(final_texto.splitlines()) == 1:
            data = [final_texto[start:end].strip() for start, end in colspecs]
            df = pd.DataFrame([data], columns=["TÍTULO", "NOME", "POSIÇÃO", "%PL", "GRUPO", "EMISSOR"])
        else:
            df = pd.read_fwf(
                StringIO(final_texto),
                colspecs=colspecs,
            )
            df.columns = ["TÍTULO", "NOME", "POSIÇÃO", "%PL", "GRUPO", "EMISSOR"]
        df = df.apply(lambda col: col.str.strip())

        def para_float_br(x):
            return float(x.replace('.', '').replace(',', '.'))

        df["POSIÇÃO"] = df["POSIÇÃO"].apply(para_float_br)
        df["%PL"] = df["%PL"].apply(para_float_br)

        #display(df)
    else:
        print("The dataset is empty.")

df = texto_memoria_para_df(texts[0])